#


In [1]:
import sqlite3
from dataclasses import dataclass
from io import BytesIO
from typing import Iterator

from PIL import Image
from IPython.display import display, clear_output
from tqdm import tqdm

In [2]:
@dataclass
class Storyboard:
    id: int
    episode: int
    frame_number: int
    subtitle: str
    picture: bytes
    role: str | None


def namedtuple_factory(cursor, row):
    return Storyboard(*row)


def db_data() -> Iterator[Storyboard]:
    try:
        with sqlite3.connect("../db/mygo.db") as conn:
            conn.row_factory = namedtuple_factory
            cursor = conn.cursor()
            res = cursor.execute("""
                            SELECT 
                                id,
                                episode,
                                frame_number,
                                subtitle,
                                picture,
                                role
                            FROM 
                                storyboards
                            WHERE
                                role IS NULL
                            ORDER BY 
                                episode, 
                                frame_number
                        """).fetchall()
            conn.commit()
            for i in res:
                yield i
    except sqlite3.OperationalError as e:
        print("Failed to open database:", e)

In [3]:
def count_remain():
    try:
        with sqlite3.connect("../db/mygo.db") as conn:
            cursor = conn.cursor()
            res = cursor.execute("""
                            SELECT 
                                COUNT(*)
                            FROM 
                                storyboards
                            WHERE
                                role IS NULL
                        """)
            return res.fetchone()[0]
    except sqlite3.OperationalError as e:
        print("Failed to open database:", e)
    return 0

In [4]:
def update(i: int, role: str):
    try:
        with sqlite3.connect("../db/mygo.db") as conn:
            cursor = conn.cursor()
            cursor.execute(
                """
                            UPDATE storyboards
                            SET role = ?
                            WHERE id = ?
                        """,
                (role, i),
            )
            conn.commit()
    except sqlite3.OperationalError as e:
        print("Failed to open database:", e)

In [8]:
cnt = count_remain()
with tqdm(total=3946) as pbar:
    for sb in db_data():
        clear_output(wait=True)
        img = Image.open(BytesIO(sb.picture)).resize((480, 270))
        display(img)
        print(f"                   {sb.subtitle}")
        res = input("""
        Q 立希 W 其他 E 初華 R 爽媽 T 海鈴 Y 喵夢 U 樂奈奶奶
        A 祥子 S 爽世 D 愛音 F 燈 G 樂奈 H 睦
        """)
        match res:
            case "a":
                role = "祥子"
            case "s":
                role = "爽世"
            case "d":
                role = "愛音"
            case "f":
                role = "燈"
            case "g":
                role = "樂奈"
            case "h":
                role = "睦"
            case "q":
                role = "立希"
            case "e":
                role = "初華"
            case "r":
                role = "爽世媽"
            case "t":
                role = "海鈴"
            case "y":
                role = "喵夢"
            case "u":
                role = "樂奈奶奶"
            case "w":
                role = "其他"

        update(sb.id, role)
        cnt += 1
        pbar.update(cnt)

  0%|                                                                                                                                                                                        | 0/3946 [00:00<?, ?it/s]
